# **Vacation Planner Data Preprocessing**

# Setup

First, we are going to install any libraries we will need in our preprocessing.

In [1]:
import pandas as pd
import zipfile
from google.colab import drive

Now, we are going to mount our drive where the downloaded files are stored.

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing of Flights

Firstly, we will perform preprocessing on our flights dataset. This one is by far the largest, and because of Datagrip/AWS's limitations, cannot be imported into the database in its current state. We will have to filter it down and reduce its size to make it appropriate for importing.

We will start by importing and extracting the zip file containing the csv files of the data, which is divided into 4 quarters.

In [3]:
# Quarter 1
zip_path = '/content/drive/My Drive/Origin_and_Destination_Survey_DB1BMarket_2022_1.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [4]:
# Quarter 2
zip_path = '/content/drive/My Drive/Origin_and_Destination_Survey_DB1BMarket_2022_2.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [5]:
# Quarter 3
zip_path = '/content/drive/My Drive/Origin_and_Destination_Survey_DB1BMarket_2022_3.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [6]:
# Quarter 4
zip_path = '/content/drive/My Drive/Origin_and_Destination_Survey_DB1BMarket_2022_4.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

Now, we have to read the csv files from each of them, while also dropping columns while we do it, so that we do not use all of our available ram.

In [7]:
to_drop = ['AirportGroup', 'WacGroup', 'TkCarrierChange', 'TkCarrierGroup',
           'OpCarrierChange', 'OpCarrierGroup', 'RPCarrier', 'TkCarrier',
           'OpCarrier', 'OriginAirportID', 'OriginAirportSeqID',
           'OriginCityMarketID', 'OriginCountry', 'OriginStateFips',
           'OriginState', 'OriginStateName', 'OriginWac', 'DestAirportID',
           'DestAirportSeqID', 'DestCityMarketID', 'DestCountry',
           'DestStateFips', 'DestState', 'DestStateName','DestWac', 'ItinID',
           'MktID', 'MktCoupons', 'Year', 'BulkFare', 'Passengers',
           'MktDistanceGroup', 'MktMilesFlown', 'NonStopMiles', 'ItinGeoType',
           'MktGeoType']

In [8]:
flights_q1_df = pd.read_csv('Origin_and_Destination_Survey_DB1BMarket_2022_1.csv')

In [9]:
flights_q1_df = flights_q1_df.drop(columns=to_drop)

In [10]:
flights_q1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6078763 entries, 0 to 6078762
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Quarter      int64  
 1   Origin       object 
 2   Dest         object 
 3   MktFare      float64
 4   MktDistance  float64
 5   Unnamed: 41  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 278.3+ MB


We can see the memory usage goes down to 278.3+ MB, which is manageable, so we will do this for the other three quarters.

In [11]:
flights_q2_df = pd.read_csv('Origin_and_Destination_Survey_DB1BMarket_2022_2.csv')

In [12]:
flights_q2_df = flights_q2_df.drop(columns=to_drop)

In [13]:
flights_q3_df = pd.read_csv('Origin_and_Destination_Survey_DB1BMarket_2022_3.csv')

In [14]:
flights_q3_df = flights_q3_df.drop(columns=to_drop)

In [15]:
flights_q4_df = pd.read_csv('Origin_and_Destination_Survey_DB1BMarket_2022_4.csv')

In [16]:
flights_q4_df = flights_q4_df.drop(columns=to_drop)

Let's drop the unnamed column, which is there likely as a result of some error when parsing the csv.

In [17]:
flights_q1_df = flights_q1_df[['Quarter', 'Origin', 'Dest', 'MktFare', 'MktDistance']]

In [18]:
flights_q2_df = flights_q2_df[['Quarter', 'Origin', 'Dest', 'MktFare', 'MktDistance']]

In [19]:
flights_q3_df = flights_q3_df[['Quarter', 'Origin', 'Dest', 'MktFare', 'MktDistance']]

In [20]:
flights_q4_df = flights_q4_df[['Quarter', 'Origin', 'Dest', 'MktFare', 'MktDistance']]

Now, we can merge all of the different dataframes together into one which we will use for out database.

In [21]:
flights_df = pd.concat([flights_q1_df, flights_q2_df, flights_q3_df, flights_q4_df], axis=0, ignore_index=True)

Now let's display the dataframe info and see the new memory usage:

In [22]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27857794 entries, 0 to 27857793
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Quarter      int64  
 1   Origin       object 
 2   Dest         object 
 3   MktFare      float64
 4   MktDistance  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.0+ GB


We can see that the memory usage is manageable at 1.0 GB, and we can turn it into a csv file which we will use in our database.

In [23]:
flights_df.to_csv('flights_raw.csv', index=False)

We now want to see if there's more details or optimizations we could gain from this dataframe. First, we'll group by origin and dest, and then take the median fare as the fare column and median distance as the distnace column.

In [24]:
flights_grouped = flights_df.groupby(['Origin', 'Dest', 'Quarter']).agg({'MktDistance': 'median', 'MktFare': 'median'}).reset_index()

flights_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269535 entries, 0 to 269534
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Origin       269535 non-null  object 
 1   Dest         269535 non-null  object 
 2   Quarter      269535 non-null  int64  
 3   MktDistance  269535 non-null  float64
 4   MktFare      269535 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 10.3+ MB


We can see that the memory usage is now 10.3+ MB, which is around 100x lower than what it was before grouping. This optimization will make our queries significantly quicker allowing our queries and database to run much quicker.

In [25]:
flights_grouped.to_csv('flights_grouped.csv', index=False)

Now, we are going to perform on analysis on the data.

In [26]:
# Print median market fare of flight to Miami

print(flights_grouped[(flights_grouped['Origin'] == 'PHL') & (flights_grouped['Dest'] == 'MIA')])

       Origin Dest  Quarter  MktDistance  MktFare
196403    PHL  MIA        1       1013.0  156.000
196404    PHL  MIA        2       1013.0  198.000
196405    PHL  MIA        3       1013.0  187.025
196406    PHL  MIA        4       1013.0  224.000


In [27]:
# Print median market fare of flight back

print(flights_grouped[(flights_grouped['Origin'] == 'MIA') & (flights_grouped['Dest'] == 'PHL')])

       Origin Dest  Quarter  MktDistance  MktFare
166381    MIA  PHL        1       1013.0  166.000
166382    MIA  PHL        2       1013.0  202.000
166383    MIA  PHL        3       1013.0  192.000
166384    MIA  PHL        4       1013.0  218.405


In [28]:
# Print median market fare of flight to Los Angeles

print(flights_grouped[(flights_grouped['Origin'] == 'PHL') & (flights_grouped['Dest'] == 'LAX')])

       Origin Dest  Quarter  MktDistance  MktFare
196283    PHL  LAX        1       2402.0  217.450
196284    PHL  LAX        2       2402.0  355.760
196285    PHL  LAX        3       2402.0  344.000
196286    PHL  LAX        4       2402.0  340.975


Let's see the minimum and maximum ticket prices

In [29]:
print(flights_grouped.sort_values(by='MktFare'))

       Origin Dest  Quarter  MktDistance  MktFare
127816    INL  JFK        1       1284.0      0.0
129765    IWD  MKE        4        400.0      0.0
199678    PIH  BIL        1        537.0      0.0
217522    ROC  ALB        4        390.0      0.0
190819    OWB  JAX        1        942.0      0.0
...       ...  ...      ...          ...      ...
33855     BQK  LIH        1       4842.0   3937.0
111685    GUM  KOA        2       3964.0   6464.0
111688    GUM  LAR        2       7279.0   7034.0
111846    GUM  RSW        2       8566.0   7359.0
202517    PNS  GUM        4       8194.0   8192.0

[269535 rows x 5 columns]


Let's drop all 0.0 MKtFare rows, since they are likely null/missing values.

In [30]:
flights_grouped = flights_grouped[flights_grouped['MktFare'] != 0]

# Preprocessing of Airbnbs

Now, we are going to perform preprocessing and analysis on our Airbnbs dataset.

In [31]:
# Delete our old dataframes to save memory
del flights_q1_df
del flights_q2_df
del flights_q3_df
del flights_q4_df
del flights_df

In [32]:
# Create the kaggle directory
!mkdir ~/.kaggle

# Read the uploaded kaggle.json file
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

# Download Airbnb
!!kaggle datasets download joebeachcapital/airbnb -f airbnb-listings.csv

# Unzip CSV
!unzip /content/airbnb-listings.csv.zip

Archive:  /content/airbnb-listings.csv.zip
  inflating: airbnb-listings.csv     


We had no problems importing this dataset into DataGrip and as it is pre-cleaned we can just directly integrate it into our database.

# Preprocessing of Destinations

Let's import and read the csv file

In [33]:
# Download Destinations
!!kaggle datasets download louise2001/us-cities -f uscities.csv

# Unzip CSV
!unzip /content/uscities.csv.zip

Archive:  /content/uscities.csv.zip
  inflating: uscities.csv            


In [34]:
destinations_df = pd.read_csv('uscities.csv')

Let's examine the current df info

In [35]:
destinations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28338 entries, 0 to 28337
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          28338 non-null  object 
 1   city_ascii    28338 non-null  object 
 2   state_id      28338 non-null  object 
 3   state_name    28338 non-null  object 
 4   county_fips   28338 non-null  int64  
 5   county_name   28338 non-null  object 
 6   lat           28338 non-null  float64
 7   lng           28338 non-null  float64
 8   population    28338 non-null  int64  
 9   density       28338 non-null  int64  
 10  source        28338 non-null  object 
 11  military      28338 non-null  bool   
 12  incorporated  28338 non-null  bool   
 13  timezone      28338 non-null  object 
 14  ranking       28338 non-null  int64  
 15  zips          28337 non-null  object 
 16  id            28338 non-null  int64  
dtypes: bool(2), float64(2), int64(5), object(8)
memory usage: 3.3+ MB


For the sake of our analysis, we will only need the 'lat', 'lng', 'city', and 'state' attributes, which we can take from the df.

In [36]:
destinations_df.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [37]:
destinations_df = destinations_df[['lat', 'lng', 'city', 'state_id']]
destinations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28338 entries, 0 to 28337
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   lat       28338 non-null  float64
 1   lng       28338 non-null  float64
 2   city      28338 non-null  object 
 3   state_id  28338 non-null  object 
dtypes: float64(2), object(2)
memory usage: 885.7+ KB


We can see how the memory usage greatly decreased, now lets drop any null values, as every column is necessary for our computations.

In [38]:
destinations_df = destinations_df.dropna()

Now, we can make a csv of our data which we will import into our database.

In [39]:
destinations_df.to_csv('destinations.csv', index=False)

# Preprocessing of Temperatures

In [42]:
# Delete old dataframe to save memory
del destinations_df

In [41]:
# Download Destinations
!!kaggle datasets download efradgamer/world-average-temperature -f Avg_World_Temp_2020.csv

unzip:  cannot find or open /content/Avg_World_Temp_2020.csv.zip, /content/Avg_World_Temp_2020.csv.zip.zip or /content/Avg_World_Temp_2020.csv.zip.ZIP.


In [51]:
temperatures_df = pd.read_csv('Avg_World_Temp_2020.csv')

Let's see the info in this dataframe

In [52]:
temperatures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  424 non-null    int64  
 1   Country     424 non-null    object 
 2   City        424 non-null    object 
 3   Jan         424 non-null    float64
 4   Feb         424 non-null    float64
 5   Mar         424 non-null    float64
 6   Apr         424 non-null    float64
 7   May         424 non-null    float64
 8   Jun         424 non-null    float64
 9   Jul         424 non-null    float64
 10  Aug         424 non-null    float64
 11  Sep         424 non-null    float64
 12  Oct         424 non-null    float64
 13  Nov         424 non-null    float64
 14  Dec         424 non-null    float64
 15  Avg_Year    424 non-null    float64
 16  Continent   424 non-null    object 
dtypes: float64(13), int64(1), object(3)
memory usage: 56.4+ KB


We can see there is a separate column for each column, we can change this to have the month be in a column and then have the average temperature for that month listed.

In [53]:
temperatures_df = temperatures_df[['Country', 'City', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Avg_Year', 'Continent']]

In [54]:
temperatures_df = temperatures_df.melt(id_vars=['Country', 'City', 'Continent', 'Avg_Year'], var_name='Month', value_name='Temperature')

In [56]:
temperatures_df.head()

,Country,City,Continent,Avg_Year,Month,Temperature
0,Algeria,Algiers,Africa,17.4,Jan,11.2
1,Algeria,Tamanrasset,Africa,21.7,Jan,12.8
2,Algeria,Reggane,Africa,28.3,Jan,16.0
3,Angola,Luanda,Africa,25.8,Jan,26.7
4,Benin,Cotonou,Africa,27.2,Jan,27.3


Let's map the months to integers to have them more easily accessed.

In [57]:
month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

temperatures_df['Month'] = temperatures_df['Month'].map(month_mapping)

In [58]:
temperatures_df.head()

,Country,City,Continent,Avg_Year,Month,Temperature
0,Algeria,Algiers,Africa,17.4,1,11.2
1,Algeria,Tamanrasset,Africa,21.7,1,12.8
2,Algeria,Reggane,Africa,28.3,1,16.0
3,Angola,Luanda,Africa,25.8,1,26.7
4,Benin,Cotonou,Africa,27.2,1,27.3


This is a much easier dataframe to work it, we can now convert it to a csv and use it in our database.

In [59]:
temperatures_df.to_csv('temperatures.csv', index=False)